In [1]:
using Calculus, Optim, LinearAlgebra, ForwardDiff

In [2]:
function krok2(funk::Function, x1::Array, k::Number)
     Grad = Calculus.gradient(funk,x1)
end

function krok3(Grad::Array,E::Number,k::Number,M::Number)
    norma = LinearAlgebra.norm(Grad)
        if norma <= E
            print("norma mniejsz/równa  E")
            return 1
        end
        if k >= M
            print("przekroczono limit iteracji")
            return 1
        end
    return 0
end
    
function krok4(funk::Function, x1::Array, k::Number, lam::Number)
    gradient = Calculus.gradient(funk,[x1[1],x1[2]])
    gradient2 = Calculus.gradient(funk,gradient)
    
    x1dokplus1 = x1[1] - (gradient2[1]+lam)^(-1) * gradient[1]
    x2dokplus1 = x1[2] - (gradient2[2]+lam)^(-1) * gradient[2]
    
    return [x1dokplus1,x2dokplus1]
end

function krok5(funkx1k1::Number,funkx1k::Number, k::Number, lam::Number)
        if funkx1k1 < funkx1k
            #krok 6
            #lam = 1/2 * lam
            #k=k+1
            #powrot do kroku 2
            return 0
        else
            #krok 7
            #lam = 2 * lam
            #powrot do kroku 4
            return 1
        end
end

krok5 (generic function with 1 method)

In [3]:
function Marquardt(funk::Function, x1::Array, E, M)
    #krok 1
    #global
    global xg = [x1[1],x1[2]]
    #global M = 100
    global k=0
    #global E=0.0000001
    global lam=10^4
    global tbc=0
    #global x1dokplus1 = [1.0,1.0,1.0,1.0]
    
    testy = 0
    
    i=0
    while i<=100
        
        @label k2
        
        #krok 2
        Grad = krok2(funk,xg,k)
        
        #krok 3
        tbc = krok3(Grad,E,k,M)
        if tbc == 1
            println("")
            print("ilosc krokow: ")
            println(testy)
            print("wartosc x y dla minimum: ")
            println(xg)
            print("minimum: ")
            return funk(xg)#Grad
        end
            
        @label k4
        #krok 4
        #x1dokplus1=krok4(funk,x1,k,lam)
        #x1dokplus1 = ForwardDiff.hessian(funk,x1)
        
        #Brakuje tu lambdy!
        x1dokplus1 = Calculus.hessian(funk, [x1[1]+0.0,x1[2]+0.0])
        
        
        #=
        
        der1 = Calculus.derivative(funk,xg)
        der1122 = Calculus.derivative(funk,der1)
        der12 = Calculus.derivative(funk,[der1[2],der1[1]])
        x1dokplus1[1] = (1 + lam) * der1122[1]
        x1dokplus1[4] = (1 + lam) * der1122[2]
        x1dokplus1[2] = der12[1]
        x1dokplus1[3] = der12[2]
        
        
        x1dominus=[1.0,2.0,3.0,4.0]
        x1dominus[1] = x1dokplus1[1]^-1
        x1dominus[2] = x1dokplus1[2]^-1
        x1dominus[3] = x1dokplus1[3]^-1
        x1dominus[4] = x1dokplus1[4]^-1
        
        
        hes1 = x1dominus
        
        =#
        
        x1dokplus1[1]*=(1+lam)
        x1dokplus1[4]*=(1+lam)
        
        hes1= x1dokplus1^-1
        xtest=[0.0,0.0]
        xtest[1] =xg[1] - hes1[1]  * Grad[1]
        xtest[2] =xg[2] - hes1[4]  * Grad[2]
        
        #krok 5
        
        
        #println(x1dokplus1)
        
        
        
        
        
        funkx1k1 = funk(xtest)
        funkx1k = funk(xg)
        
        #print("funk do xtest = ")
        #println(funkx1k1)
        #print("funk do x = ")
        #println(funkx1k)
        
        #wracamy = krok5(funkx1k1,funkx1k,k,lam)
        testy+=1
        if testy >=100
            #println("Przekroczono limit iteracji.")
            print("ilosc krokow: ")
            println(testy)
            print("wartosc x y dla minimum: ")
            println(xg)
            return funk(x1)#Grad
            #return Grad
        end
        if funkx1k1 < funkx1k
            lam = 1/2 * lam
            k=k+1
            xg=xtest
            #print("skok do 2   ")
            #print(k)
            @goto k2
        else
            lam = 2 * lam
            #lam = 1/2 * lam
            #print("skok do 4    ")
            #print(lam)
            @goto k4
        end
        
        
        
        
        
        #=
        #krok5
        if funk(xtest) > funk(x1)
            #krok6(4)
            lam = 2 * lam
            print("skok do 4   ")
            print(k)
            #@goto k4
        else
            #krok7(5)
            k=k+1
            x1=xtest
            lam = 1/2 * lam
            print("skok do 2    ")
            print(lam)
            @goto k2
        end
            =#
        
        
        #=
        =#
        
        i=i+1
        #k=k+1
    end
        return "Wystapil jakis blad. funkcja nigdy nie powinna tu skonczyc"
end

Marquardt (generic function with 1 method)

In [4]:
function losowa(x)
   return  sin(x[1]) + cos(x[2])
end

Marquardt(losowa,[10.0,110.0],0.0001,100)
#jeszczeraz(losowa,[121.0,84.0])

norma mniejsz/równa  E
ilosc krokow: 31
wartosc x y dla minimum: [10.9957, 109.956]
minimum: 

-1.9999999955804921